In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#from lxml import html

# def getGradeSource(URL):
    arrTotal = []

    quote_page = URL
    #URL of gradesource website that needs to be scraped
    page = requests.get(quote_page)
    #making a request
    soup = BeautifulSoup(page.content, 'html.parser')
    #cleaning the html
    
    table = soup.find_all('tr')
    #retrieving all the table
    arr = []

    for i in range(len(table)):
        arr.append(table[i].find_all('td'))

    for i in range(len(arr)):
        if i >= 7 and i < len(arr):     
            arrHolder = []
            for k in range(len(arr[i])):
                if arr[i][k].find("b") != None:
                    arrHolder.append(arr[i][k].find("b").text) # getting the course id
                elif arr[i][k].find("font") != None:
                    arrHolder.append(arr[i][k].find("font").text) # getting the rank
            arrTotal.append(arrHolder)
    return arrTotal

In [44]:
def getGradeSourceGrades(URL):
    arrTotal = []

    quote_page = URL
    #URL of gradesource website that needs to be scraped
    page = requests.get(quote_page)
    #making a request
    soup = BeautifulSoup(page.content, 'html.parser')
    #cleaning the html
    
    table = soup.find_all('tr')
    #retrieving all the table
    arr = []

    for i in range(len(table)):
        arr.append(table[i].find_all('td'))

    arrHolder = {}
    for i in range(len(arr)):
        if i >= 8 and i < len(arr):     
            if arr[i][0].find("b") != None:

                arrHolder[arr[i][0].find("b").text] = {}
                
                if arr[i][len(arr[i])-1].find("b") != None:
               
                    arrHolder[arr[i][0].find("b").text]["Grade"] = arr[i][len(arr[i])-1].find("b").text  # getting the course id
              
                elif arr[i][len(arr[i])-1].find("font") != None:
                
                    arrHolder[arr[i][0].find("b").text]["Grade"] = arr[i][len(arr[i])-1].find("font").text # getting the rank
           
                arrHolder[arr[i][0].find("b").text]["Rank"] = arr[i][len(arr[i])-2].find("font").text + "/" + str(len(arr)-4)
            elif arr[i][0].find("font") != None:

                arrHolder[arr[i][0].find("font").text] = {}
                
                if arr[i][len(arr[i])-1].find("b") != None:

                    arrHolder[arr[i][0].find("font").text]["Grade"] = arr[i][len(arr[i])-1].find("b").text # getting the course id

                elif arr[i][len(arr[i])-1].find("font") != None:

                    arrHolder[arr[i][0].find("font").text]["Grade"] = arr[i][len(arr[i])-1].find("font").text # getting the rank
                
                arrHolder[arr[i][0].find("font")]["Rank"] = str(arr[i][len(arr[i])-2].find("font").text) + "/" + str(len(arr)-4)
    #arrTotal.append(arrHolder)
    return arrHolder

In [45]:
CSE110 = getGradeSourceGrades('http://www.gradesource.com/reports/7/29889/cs126276.html')
cse15L = getGradeSourceGrades('http://www.gradesource.com/reports/7/29142/categoryscores.html')
#Change the url to a url with gradesource

In [46]:
#prints out the the key and value
#for key, value in CSE110.items():
#    print(key, value)

8605 {'Grade': '94.12%', 'Rank': '121/247'}
0233 {'Grade': '92.37%', 'Rank': '166/247'}
4605 {'Grade': '95.15%', 'Rank': '96/247'}
7182 {'Grade': '87.81%', 'Rank': '216/247'}
6671 {'Grade': '95.91%', 'Rank': '76/247'}
4512 {'Grade': '94.36%', 'Rank': '118/247'}
0273 {'Grade': '93.35%', 'Rank': '141/247'}
7181 {'Grade': '91.70%', 'Rank': '181/247'}
7373 {'Grade': '96.07%', 'Rank': '69/247'}
3243 {'Grade': '98.11%', 'Rank': '17/247'}
8368 {'Grade': '94.32%', 'Rank': '119/247'}
1451 {'Grade': '93.76%', 'Rank': '130/247'}
4584 {'Grade': '94.52%', 'Rank': '115/247'}
7643 {'Grade': '96.25%', 'Rank': '66/247'}
6106 {'Grade': '94.38%', 'Rank': '116/247'}
2478 {'Grade': '95.33%', 'Rank': '87/247'}
8681 {'Grade': '88.06%', 'Rank': '214/247'}
6234 {'Grade': '97.80%', 'Rank': '21/247'}
2373 {'Grade': '95.33%', 'Rank': '86/247'}
4368 {'Grade': '96.59%', 'Rank': '55/247'}
8221 {'Grade': '93.49%', 'Rank': '137/247'}
3259 {'Grade': '98.13%', 'Rank': '16/247'}
0216 {'Grade': '94.69%', 'Rank': '111/247'

In [47]:
#this code is used for formatting into a json 
import json
courseList = json.dumps(CSE110, indent=4)
courseList = courseList.replace("/", "／")
courseList = courseList.replace(".", "．")

jsonDict = json.loads(courseList)

print(courseList)
jsonDict = json.loads(courseList)

242
{
    "CSE110_Sp18": {
        "8605": {
            "Grade": "94．12%",
            "Rank": "121／247"
        },
        "0233": {
            "Grade": "92．37%",
            "Rank": "166／247"
        },
        "4605": {
            "Grade": "95．15%",
            "Rank": "96／247"
        },
        "7182": {
            "Grade": "87．81%",
            "Rank": "216／247"
        },
        "6671": {
            "Grade": "95．91%",
            "Rank": "76／247"
        },
        "4512": {
            "Grade": "94．36%",
            "Rank": "118／247"
        },
        "0273": {
            "Grade": "93．35%",
            "Rank": "141／247"
        },
        "7181": {
            "Grade": "91．70%",
            "Rank": "181／247"
        },
        "7373": {
            "Grade": "96．07%",
            "Rank": "69／247"
        },
        "3243": {
            "Grade": "98．11%",
            "Rank": "17／247"
        },
        "8368": {
            "Grade": "94．32%",
            "Rank": "119／247

In [40]:
from firebase import firebase

In [41]:

#This code is intended to send the dictionary to firebase
firebase = firebase.FirebaseApplication('https://YourFirebaseDB.firebaseio.com/')

result = firebase.patch('/GradeSource', { "NameOfCourse": jsonDict})